In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

random_seed = 123
np.random.seed(random_seed)

# 1. Reading the data

Histone modifications play an important role in affecting gene regulation. Specific histone modifications at specific locations in or near the gene can alter the expression of genes. Predicting gene expression from histone modification signals is a widely studied research topic.

In this competition you will predict gene expression levels (low=0, high=1) based on the presence of histone modifications at specific locations in the gene. You will try to find the model that learns the true underlying model best.

For each gene a region of 10.000bp around the transcription start site of the gene is extracted (5000bp upstream and 5000bp downstream). This region is binned in 100 bins of 100bp. For each bin five core histone modification marks are counted [1].

The dataset is compiled from the "E047" (Primary T CD8+ naive cells from peripheral blood) celltype from Roadmap Epigenomics Mapping Consortium (REMC) database.

[1] Kundaje, A. et al. Integrative analysis of 111 reference human epige-
nomes. Nature, 518, 317–330, 2015.


We start by loading the Pandas library and reading the datasets into Pandas DataFrames:

In [ ]:
import pandas as pd

train = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data_test.csv")

Let's look at the first 5 rows of the trainset:

In [ ]:
train.head(5)

There is a column called `GeneId` that identifies the gene. This column should not be used as a feature. 

The label for each datapoint is in the `Label` column.

In [ ]:
train_ids = train.pop("GeneId")
train_labels = train.pop("Label")

Now `train` contains the feature columns only.

Let's look at the number datapoints in each class:

In [ ]:
train_labels.value_counts()

Let's look at `test`:

In [ ]:
test.head(5)

This is a blind test so the `Label` column is not available in the testset. The testset does contain the `GeneId` column that is needed to send your predictions to the Kaggle website.


In [ ]:
test_index_col = test.pop("GeneId")

We can get some general statistics about the trainset using the DataFrame `.describe()` function:

In [ ]:
train.describe().transpose()

# 2. Data pre-processing




We can use the Pandas `boxplot()` function to plot the feature values:

In [ ]:
plt.figure(figsize=(22,8))
train.boxplot()
plt.show()

Let's plot these for each hisotone mark:

In [ ]:
marks = {}
for m in train.columns:
  marks[m.split("_")[0]] = True
marks = list(marks.keys())
marks

In [ ]:
for mark in marks:
  cols = []
  for m in train.columns:
    if mark in m:
      cols.append(m)
  plt.figure(figsize=(22,8))    
  train[cols].boxplot()
  plt.show()

In this case I choose scaling the features to [0,1]:

In [ ]:
from sklearn import preprocessing

scaler_minmax = preprocessing.MinMaxScaler()
scaler_minmax.fit(train)
train_norm = pd.DataFrame(scaler_minmax.transform(train),columns=train.columns)
train_norm.head()

In [ ]:
for mark in marks:
  cols = []
  for m in train_norm.columns:
    if mark in m:
      cols.append(m)
  plt.figure(figsize=(22,8))    
  train_norm[cols].boxplot()
  plt.show()

# 3. Building a decision tree

The scikit-learn `DecisionTreeClassifier` class computes a decision tree predictive model from a dataset. 

To get all the options for learning you can simply type: 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
help(DecisionTreeClassifier)

You notice that there are many (hyper)parameters to set. These influence the complexity of the model. An important such parameter is the `max_depth` that set a limit on how deep a decision tree can become. 

Let's create a decision tree model with `max_depth=3`:

In [ ]:
cls_DT = DecisionTreeClassifier(max_depth=3)

This creates a decision tree model with default values for the other hyperparameters:

In [ ]:
cls_DT

Now we can fit the trainset in just one line of code:

In [ ]:
cls_DT.fit(train_norm,train_labels)

Let's see how well the decision tree performs on the trainset:

In [ ]:
from sklearn.metrics import accuracy_score

predictions = cls_DT.predict(train_norm)
print("Accuracy: %f"%(accuracy_score(predictions, train_labels)))

These are the predictions for the first 100 data points:

In [ ]:
predictions[0:100]

For the Kaggle competition your predictions are not evaluated by accuracy, but by log-loss:

$$ - \frac{1}{N} \sum_{i=1}^N [y_{i} \log \, p_{i} + (1 - y_{i}) \log \, (1 - p_{i})],$$

where $N$ is the number of datapoints, $y_i$ is the label of datapoint $i$, and $p_i$ is the prediction of the model expressed as a probability.

Let's compute the log-loss:


In [ ]:
from sklearn.metrics import log_loss

print("Log-loss: %f"%log_loss(train_labels,predictions))

This is a very high log-loss (note that we want the log-loss the be as close to zero as possible).

The reason is that we don't actually provide probabilities but just the classes 0 and 1. Any error will be punished with a very high contribution to the log-loss. Therefor, we should predict class probabilities rather than just classes. 

Decision trees allow us to do that:

In [ ]:
predictions = cls_DT.predict_proba(train_norm)

print("Log-loss: %f"%log_loss(train_labels,predictions))

Now the log-loss is much smaller.

The following code plots the fitted decision tree `cls` as a `tree.png` file:

In [ ]:
"""
from sklearn import tree
from io import StringIO
from IPython.display import Image, display
import pydotplus

out = StringIO()
tree.export_graphviz(cls_DT, out_file=out)
graph=pydotplus.graph_from_dot_data(out.getvalue())
graph.write_png("tree.png")
"""

# 4. Kaggle evaluation

Let's make a Kaggle submission:

In [ ]:
#code for submission
predictions_test = cls_DT.predict_proba(test)

out_tmp = pd.DataFrame()
out_tmp["GeneId"] = test_index_col
out_tmp["eyeDetection"] = predictions_test[:,1]
out_tmp.to_csv("submission_DT.csv",index=False)

How does the model perform on the testset?

# 5. Hyperparamters

For our first submission we set the hyperparameter `max_depth=3`. Other values might result in lower log-loss on the testset. 

Since we don't have the testset labels we can only check this on the public leaderboard, which we can/should not do!

So, we need to create our own testset (**not seen during training!**) with known class labels.

Scikit-learn offers many options to do this. One of them is the `train_test_split` function:

In [ ]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(train_norm,train_labels,
                                                  test_size=.2, random_state=random_seed)

#train fold
print(train_X.shape)
print(train_y.shape)
#validation fold
print(val_X.shape)
print(val_y.shape)

Fit a decision tree model with `max_depth=14` default paramters on the `train_X` data set.

In [ ]:
#solution

What is the accuracy and log-loss on `train_X`? 

In [ ]:
#solution

What is the accuracy and log-loss on `val_X`?

In [ ]:
#solution

What do you see?


The following code evaluates different values for this hyperparameter.

In [ ]:
for maxdepth in range(1,20,1):
    cls = DecisionTreeClassifier(max_depth=maxdepth)
    cls.fit(train_X,train_y)
    predictions_train = cls.predict(train_X)
    predictions_val = cls.predict(val_X)
    predictions_train_prob = cls.predict_proba(train_X)[:,1]
    predictions_val_prob = cls.predict_proba(val_X)[:,1]
    print("%i (%f) %f (%f) %f"%(maxdepth,
                                accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y),
                                log_loss(train_y,predictions_train_prob),log_loss(val_y,predictions_val_prob)))

What do you see?

So, we have split the data into a train- and validationset. We can of course split the data in many different ways (different random seeds) resulting in different train- and validationsets.

Let's try 5 different random seeds:

In [ ]:
for run in range(5):
  train_X, val_X, train_y, val_y = train_test_split(train_norm,train_labels,
                                                  test_size=.8, random_state=run)
  min_m = 100
  best = None
  for maxdepth in range(1,20,1):
      cls = DecisionTreeClassifier(max_depth=maxdepth)
      cls.fit(train_X,train_y)
      predictions_val_prob = cls.predict_proba(val_X)[:,1]
      m = log_loss(val_y,predictions_val_prob)
      if m < min_m:
        min_m = m
        best = maxdepth
  print("%i %f"%(best,min_m))

What do you see?

The solution is to run several train-validations splits and average the performance.

One popular method is cross-validation that uses each datapoint once as a testpoint.

It works as follows:
<br/>
<br/>
<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png"/>
<br/>
<br/>

It is easy to run this in Scikit-learn:



In [ ]:
from sklearn.model_selection import cross_val_predict

for maxdepth in range(1,10,1):
    cls = DecisionTreeClassifier(max_depth=maxdepth)
    predictions = cross_val_predict(cls,train_norm,train_labels,
                                    cv=10,
                                    method="predict_proba")
    print("%i %f"%(maxdepth,log_loss(train_labels,predictions[:,1])))

We can do this in two lines of code with the `GridSearchCV` module:


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth':range(1,10)
    }

GSCV = GridSearchCV(cls_DT, params,
                    cv=10,
                    scoring="neg_log_loss",
                    verbose=1).fit(train_norm,train_labels)

print(GSCV.best_estimator_)
print(GSCV.best_score_)

Play with the hyperparameters in a Template notebook and make some Kaggle submissions.

# 5. Ensemble learning: bagging

We have seen that bias and variance play an important role in Machine Learning. 

Let's first see what bagging can do for our dataset. 

In [ ]:
from sklearn.ensemble import BaggingClassifier

cls = BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=random_seed)
                                                            
cls.fit(train_X,train_y)
predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)
print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))
predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)
print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,1]),log_loss(val_y,predictions_val_prob[:,1])))

With the `RandomForestClassifier` the variance of the decision tree is reduced also by selecting features for decision tree contruction at random. Let's see how far we get with default hyperparameter values.   

In [ ]:
from sklearn.ensemble import RandomForestClassifier

cls = RandomForestClassifier(random_state=random_seed)

cls.fit(train_X,train_y)
predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)
print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))
predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)
print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,1]),log_loss(val_y,predictions_val_prob[:,1])))

# 6. Ensemble learning: boosting

How about the `GradientBoostingClassifier`?

In [ ]:
#solution